In [1]:
import warnings
warnings.filterwarnings("ignore")
import re
import os
import random
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import numpy as np
import string
from keras.models import model_from_json 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

In [2]:
with open('tokenizer_t.pkl','rb') as f:
    tokenizer_text = pickle.load(f)

In [3]:
with open('tokenizer_s.pkl','rb') as f:
    tokenizer_sentiment = pickle.load(f)

In [4]:
# Loading JSON file
from keras.models import model_from_json  
json_file = open("model_final_1.json", 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
  
# Loading weights
loaded_model.load_weights("model_final_1.h5")

In [ ]:
def convert_index_to_text(data):
    """This function reconvert the starting and ending index back to text"""
  
     text_col = data[0]   
     start_col = data[1] 
     end_col = data[2]
     end_col =end_col + 1
     text_col = text_col.split()
     selected_text_col = text_col[start_col:end_col]
     selected_text_col = " ".join(selected_text_col)
     return selected_text_col

In [6]:
def jaccard(x): 
    str1 = str(x[0])
    str2 = str(x[1])
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

## Final functions

In [11]:
def final_func_1(X):
    '''This function takes a text(string) and sentiment as input and 
       returns selected_text(keywords) as output'''
    df_text = inputs['text'].values
    df_sentiment = inputs['sentiment'].values

    df_text = tokenizer_text.texts_to_sequences(df_text)
    df_text = pad_sequences(df_text,32,padding='post')

    df_sentiment = tokenizer_sentiment.texts_to_sequences(df_sentiment)
    df_sentiment = pad_sequences(df_sentiment,1,padding='post')
    prediction = loaded_model.predict([df_text,df_sentiment])
    X['start'],X['end'] = (abs(prediction[:,0])),(abs(prediction[:,1]))
    X['start'] = X['start'].astype('int')
    X['end'] = X['end'].astype('int')
    X['predicted_text'] = X[['text','start','end']].progress_apply(lambda i : convert_index_to__text(i),axis=1)
    X = X.drop(['start','end'],axis=1)
    return X

In [12]:
def final_func_2(result):
    """This function will produce jaccard score by comparing selected_text and predicted_text"""
    
    result['jaccard_score'] = result[['predicted_text','selected_text']].apply(jaccard,axis=1)
    score = np.mean(result['jaccard_score'])
    return score

## Example 1

In [13]:
text = 'shanghai is also really exciting precisely skyscrapers galore good tweeps in china sh bj'
sentiment = 'positive'
selected_text = 'really exciting'
inputs = pd.DataFrame([[text, selected_text,sentiment]],columns=['text','selected_text', 'sentiment'],dtype='str')
                                       
inputs

,text,selected_text,sentiment
0,shanghai is also really exciting precisely sky...,really exciting,positive


In [14]:
result = final_func_1(inputs)
result

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 969.78it/s]


,text,selected_text,sentiment,predicted_text
0,shanghai is also really exciting precisely sky...,really exciting,positive,really exciting


In [15]:
print('The jaccard score is :',final_func_2(result))

The jaccard score is : 1.0


## Example 2

In [25]:
text = 'my back hurts really bad'
sentiment = 'negative'
selected_text = 'really bad'
inputs2 = pd.DataFrame([[text, selected_text,sentiment]],columns=['text','selected_text', 'sentiment'],dtype='str')
                                       
inputs2

,text,selected_text,sentiment
0,my back hurts really bad,really bad,negative


In [26]:
result2 = final_func_1(inputs2)
result2

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 496.19it/s]


,text,selected_text,sentiment,predicted_text
0,my back hurts really bad,really bad,negative,really bad


In [27]:
print('The jaccard score is :',final_func_2(result2))

The jaccard score is : 1.0


## Model Deployment

For Model Deployment video [click here](https://www.youtube.com/watch?v=7kMUtQmw2BU)